In [1]:
import pandas as pd

# Load the Excel file
file_path = 'Evaluation Result - Clean.xlsx'
df = pd.read_excel(file_path)
df_filtered = df[df['evaluator_id'] != 2]

# Filter for evaluators 4, 5, and 6
evaluators_filter = df_filtered['evaluator_id'].isin([4, 5, 6])

# Filter for specific project names
projects_filter = df_filtered['project_name'].isin(['alfred', 'camperplus', 'recycling', 'nsf', 'datahub'])

# Apply both filters
filtered_df = df_filtered[evaluators_filter & projects_filter]

filtered_df['rank'] = filtered_df.groupby(['project_name', 'evaluator_id','criteria_id'])['score'].rank(method='dense', ascending=False)


# Display the first few rows of the dataframe to understand its structure
df_filtered

C:\Users\guntu\AppData\Local\Temp\ipykernel_27444\737920700.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['rank'] = filtered_df.groupby(['project_name', 'evaluator_id','criteria_id'])['score'].rank(method='dense', ascending=False)


,id,evaluator_id,project_name,method,criteria_id,score
16,17,4,alfred,gpt-3.5,1,2
17,18,4,alfred,gpt-4,1,4
18,19,4,alfred,llama2,1,1
19,20,4,alfred,mixtral,1,3
20,21,4,alfred,gpt-3.5,2,2
...,...,...,...,...,...,...
251,292,6,recycling,mixtral,1,3
252,293,6,recycling,gpt-3.5,2,1
253,294,6,recycling,gpt-4,2,4
254,295,6,recycling,llama2,2,1


In [2]:
# Group the data by 'method' and 'criteria_id' and calculate the average score for each group
avg_scores = df_filtered.groupby(['method', 'criteria_id'])['score'].mean().unstack().reset_index()
avg_scores.columns = ['Method', 'Completeness (Avg Score)', 'Correctness (Avg Score)']

avg_scores

,Method,Completeness (Avg Score),Correctness (Avg Score)
0,gpt-3.5,2.766667,2.433333
1,gpt-4,4.000000,3.700000
2,llama2,2.333333,1.833333
3,mixtral,3.000000,2.800000


In [3]:
sorted_df = filtered_df[(filtered_df['evaluator_id'] == 4) & 
                        (filtered_df['project_name'] == 'alfred') & 
                        (filtered_df['criteria_id'] == 1)].sort_values(by='method', ascending=False)

sorted_df

,id,evaluator_id,project_name,method,criteria_id,score,rank
19,20,4,alfred,mixtral,1,3,2.0
18,19,4,alfred,llama2,1,1,4.0
17,18,4,alfred,gpt-4,1,4,1.0
16,17,4,alfred,gpt-3.5,1,2,3.0


In [4]:
from scipy.stats import spearmanr, kendalltau

# Pivot the DataFrame to have evaluators' ranks in the same row for comparison
pivot_df = filtered_df.pivot_table(index=['project_name', 'method', 'criteria_id'],
                                   columns='evaluator_id',
                                   values='rank')


In [5]:
pivot_df

evaluator_id                        4    5    6
project_name method  criteria_id               
alfred       gpt-3.5 1            3.0  2.0  2.0
                     2            3.0  1.0  3.0
             gpt-4   1            1.0  1.0  1.0
                     2            1.0  3.0  2.0
             llama2  1            4.0  3.0  3.0
                     2            4.0  4.0  4.0
             mixtral 1            2.0  2.0  1.0
                     2            2.0  2.0  1.0
camperplus   gpt-3.5 1            3.0  1.0  2.0
                     2            2.0  1.0  1.0
             gpt-4   1            2.0  2.0  1.0
                     2            1.0  2.0  1.0
             llama2  1            1.0  3.0  3.0
                     2            1.0  2.0  2.0
             mixtral 1            4.0  4.0  3.0
                     2            3.0  3.0  2.0
datahub      gpt-3.5 1            3.0  2.0  4.0
                     2            4.0  3.0  4.0
             gpt-4   1            2.0  1.0  1.0
                     2            1.0  1.0  1.0
             llama2  1            4.0  3.0  3.0
                     2            3.0  3.0  3.0
             mixtral 1            1.0  1.0  2.0
                     2            2.0  2.0  2.0
nsf          gpt-3.5 1            3.0  2.0  2.0
                     2            3.0  3.0  3.0
             gpt-4   1            2.0  2.0  1.0
                     2            2.0  2.0  1.0
             llama2  1            4.0  3.0  3.0
                     2            4.0  3.0  3.0
             mixtral 1            1.0  1.0  2.0
                     2            1.0  1.0  2.0
recycling    gpt-3.5 1            3.0  2.0  3.0
                     2            3.0  2.0  3.0
             gpt-4   1            1.0  1.0  1.5
                     2            2.0  1.0  2.0
             llama2  1            4.0  3.0  4.0
                     2            4.0  2.0  3.0
             mixtral 1            2.0  2.0  3.0
                     2            1.0  2.0  2.5

In [6]:
# Calculate Spearman's and Kendall's correlation coefficients between evaluator pairs
evaluator_pairs = [(4, 5), (4, 6), (5, 6)]
for pair in evaluator_pairs:
    spearman_corr, spearman_p = spearmanr(pivot_df[pair[0]], pivot_df[pair[1]], nan_policy='omit')
    kendall_corr, kendall_p = kendalltau(pivot_df[pair[0]], pivot_df[pair[1]], nan_policy='omit')
    
    print(f"Evaluators {pair[0]} and {pair[1]} Spearman's correlation: {spearman_corr:.3f} (p-value: {spearman_p:.3f})")
    print(f"Evaluators {pair[0]} and {pair[1]} Kendall's tau: {kendall_corr:.3f} (p-value: {kendall_p:.3f})")

Evaluators 4 and 5 Spearman's correlation: 0.594 (p-value: 0.000)
Evaluators 4 and 5 Kendall's tau: 0.532 (p-value: 0.000)
Evaluators 4 and 6 Spearman's correlation: 0.666 (p-value: 0.000)
Evaluators 4 and 6 Kendall's tau: 0.549 (p-value: 0.000)
Evaluators 5 and 6 Spearman's correlation: 0.594 (p-value: 0.000)
Evaluators 5 and 6 Kendall's tau: 0.505 (p-value: 0.000)
